<a href="https://colab.research.google.com/github/MindXpansion/Pandas-Cheat-Sheet-for-Data-Analysis/blob/main/task_manager_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# /Users/bear/task_manager_colab.py
"""
Task Manager w/Authentication - Google Colab Edition
====================================================

This app lets users track & manage their tasks securely thru Google Colab.
Users can register, login, & handle all their task needs - adding, viewing,
marking complete, & removing tasks as needed. Each user's tasks are kept
separate & secure on Google Drive.

At the end of the day... this is a simple but robust solution for keeping
your tasks organized & accessible from anywhere.

Setup is straightforward:
1. Run in Google Colab
2. Connect to your Google Drive when prompted
3. The app creates a 'task_manager' folder in your Drive automatically
"""

import os
import json
import hashlib
import time
import csv
from datetime import datetime, timedelta, date
import getpass
from google.colab import drive
import IPython.display as display
import pytz

class DateTimeManager:
    """Handles all datetime operations consistently throughout the application."""

    def __init__(self, timezone='America/Denver'):
        """Initialize with a default timezone.

        Args:
            timezone: IANA timezone string (default: America/Denver)
        """
        self.timezone = pytz.timezone(timezone)

    def now(self):
        """Get the current datetime in the configured timezone.

        Returns:
            Timezone-aware datetime object
        """
        return datetime.now(self.timezone)

    def today(self):
        """Get the current date in the configured timezone.

        Returns:
            Date object for today
        """
        return self.now().date()

    def format_datetime(self, dt=None):
        """Format a datetime for storage in a standard format.

        Args:
            dt: Datetime to format (default: current datetime)

        Returns:
            Formatted datetime string
        """
        if dt is None:
            dt = self.now()
        return dt.strftime('%Y-%m-%d %H:%M:%S')

    def format_date(self, d=None, format='%m-%d-%Y'):
        """Format a date for display in a standard format.

        Args:
            d: Date to format (default: today)
            format: Format string to use (default: %m-%d-%Y)

        Returns:
            Formatted date string
        """
        if d is None:
            d = self.today()
        if isinstance(d, datetime):
            d = d.date()
        return d.strftime(format)

    def parse_date(self, date_str):
        """Parse a date string in multiple formats.

        Args:
            date_str: Date string to parse

        Returns:
            Date object or None if parsing fails
        """
        if not date_str or date_str == 'Not set':
            return None

        # Try multiple formats
        formats = ['%m-%d-%Y', '%B %d, %Y', '%b %d, %Y', '%Y-%m-%d']

        for fmt in formats:
            try:
                return datetime.strptime(date_str, fmt).date()
            except ValueError:
                continue

        return None

    def get_date_status(self, date_str):
        """Determine if a date is in the past, today, or future.

        Args:
            date_str: Date string to check

        Returns:
            String status: 'past', 'today', 'future', or 'unknown'
        """
        date_obj = self.parse_date(date_str)

        if date_obj is None:
            return 'unknown'

        today = self.today()

        if date_obj < today:
            return 'past'
        elif date_obj == today:
            return 'today'
        else:
            return 'future'

    def get_relative_date(self, days_offset=0):
        """Get a date relative to today.

        Args:
            days_offset: Number of days from today (negative for past)

        Returns:
            Date object
        """
        return self.today() + timedelta(days=days_offset)

    def format_relative_date(self, days_offset=0, format='%m-%d-%Y'):
        """Get a formatted date string relative to today.

        Args:
            days_offset: Number of days from today (negative for past)
            format: Format string to use

        Returns:
            Formatted date string
        """
        relative_date = self.get_relative_date(days_offset)
        return self.format_date(relative_date, format)

class TaskManager:
    """Main TaskManager class - handles the heavy lifting for users & tasks."""

    def __init__(self, timezone='America/Denver'):
        """Gets everything setup & ready to roll."""
        # Initialize datetime manager
        self.datetime_mgr = DateTimeManager(timezone)

        # Setup our folders - keeping it organized
        self.base_dir = '/content/drive/MyDrive/task_manager'

        # Our data files - keeping it simple
        self.users_file = os.path.join(self.base_dir, 'users.json')
        self.tasks_file = os.path.join(self.base_dir, 'tasks.json')
        self.csv_log = os.path.join(self.base_dir, 'task_log.csv')

        # Connect to Google Drive - but don't force it yet
        self._check_drive_connected()

        # Track who's logged in right now
        self.current_user = None
        self.is_logged_in = False

        # Define categories
        self.categories = ["Work", "Personal", "Study", "Health", "Finance", "Other"]

    def _check_drive_connected(self):
        """Check if Google Drive is connected.

        Returns:
            bool: True if connected, False otherwise
        """
        return os.path.exists('/content/drive/MyDrive')

    def _connect_to_google_drive(self, force_auth=False):
        """Connects to Google Drive if not already connected.

        Args:
            force_auth: Force reconnection attempt even if already connected

        Returns:
            bool: True if connection successful, False otherwise
        """
        try:
            # Check if already mounted by looking for the drive path
            if os.path.exists('/content/drive/MyDrive') and not force_auth:
                print("Google Drive is already connected.")
                return True

            # If not mounted or force_auth is True, attempt to mount
            print("Connecting to Google Drive...")
            print("Please follow the authentication steps in the popup window if prompted.")

            # Mount the drive
            drive.mount('/content/drive', force_remount=force_auth)

            if os.path.exists('/content/drive/MyDrive'):
                print("Successfully connected to Google Drive!")

                # Create task_manager folder if it doesn't exist
                if not os.path.exists(self.base_dir):
                    os.makedirs(self.base_dir)
                    print(f"Created task_manager folder in your Google Drive.")

                # Initialize files
                self._initialize_files()

                return True
            else:
                print("Failed to connect to Google Drive. Please try manual authentication.")
                return False

        except Exception as e:
            print(f"Error connecting to Google Drive: {e}")
            print("You may need to manually authenticate in the pop-up window.")
            return False

    def _initialize_files(self):
        """Creates our data files if they don't exist yet."""
        if not os.path.exists('/content/drive/MyDrive'):
            print("Google Drive is not connected. Can't initialize files.")
            return

        # Users file check
        if not os.path.exists(self.users_file):
            with open(self.users_file, 'w') as f:
                json.dump({}, f)

        # Tasks file check
        if not os.path.exists(self.tasks_file):
            with open(self.tasks_file, 'w') as f:
                json.dump({}, f)

        # CSV log check
        if not os.path.exists(self.csv_log):
            with open(self.csv_log, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['task_id', 'username', 'description', 'category', 'priority', 'due_date',
                                'created_at', 'completed_at', 'completed_by', 'status'])

    def _hash_password(self, password):
        """Secures passwords so they're not stored as plain text.

        Let's be real... security matters, even in simple apps.

        Args:
            password: Plain text password to secure

        Returns:
            Hashed version of the password
        """
        # Simple but effective for our needs
        return hashlib.sha256(password.encode()).hexdigest()

    def register(self, username, password):
        """Sets up a new user account.

        Args:
            username: Desired username
            password: Chosen password

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to register."

        # Quick validation - no empty fields allowed
        if not username or not password:
            return False, "C'mon now... username & password can't be empty!"

        # Load existing users
        try:
            with open(self.users_file, 'r') as f:
                users = json.load(f)
        except json.JSONDecodeError:
            # If the file is corrupt or empty, start fresh
            users = {}
        except Exception as e:
            return False, f"Error reading user data: {e}"

        # Make sure username isn't taken
        if username in users:
            return False, "That username is already taken. Try another one."

        # Add the new user w/secure password
        users[username] = {
            'password': self._hash_password(password),
            'created_at': self.datetime_mgr.format_datetime()
        }

        # Save updated users data
        try:
            with open(self.users_file, 'w') as f:
                json.dump(users, f)
        except Exception as e:
            return False, f"Error saving user data: {e}"

        # Set up empty task list for the new user
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            tasks = {}
        except Exception as e:
            return False, f"Error reading task data: {e}"

        tasks[username] = []

        try:
            with open(self.tasks_file, 'w') as f:
                json.dump(tasks, f)
        except Exception as e:
            return False, f"Error saving task data: {e}"

        return True, f"All set! You're registered & ready to go, {username}."

    def login(self, username, password):
        """Gets user logged into the system.

        Args:
            username: User's username
            password: User's password

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to login."

        # Basic validation
        if not username or not password:
            return False, "Username & password are both required... try again."

        # Load user data
        try:
            with open(self.users_file, 'r') as f:
                users = json.load(f)
        except json.JSONDecodeError:
            return False, "No users found. You'll need to register first."
        except Exception as e:
            return False, f"Error reading user data: {e}"

        # Check if user exists & password is correct
        if username not in users:
            return False, "Username not found. You'll need to register first."

        if users[username]['password'] != self._hash_password(password):
            return False, "Password doesn't match... try again."

        # Set current user & login status
        self.current_user = username
        self.is_logged_in = True

        return True, f"Welcome back, {username}! Your tasks are ready."

    def logout(self):
        """Logs current user out of the system."""
        if self.is_logged_in:
            username = self.current_user
            self.current_user = None
            self.is_logged_in = False
            return f"You're logged out, {username}. Thanks for stopping by!"
        return "No one's logged in right now."

    def add_task(self):
        """Interactive task addition with prompts for all fields.

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to add task."

        if not self.is_logged_in:
            return False, "You need to login first to add tasks."

        # Get task description
        description = input("What's the task? ")
        if not description:
            return False, "Task description can't be empty... what do you need to do?"

        # Get category
        print("\nCategory options:")
        for i, category in enumerate(self.categories):
            print(f"{i+1}. {category}")

        category_choice = input(f"Choose category (1-{len(self.categories)}): ")
        try:
            category_idx = int(category_choice) - 1
            if 0 <= category_idx < len(self.categories):
                category = self.categories[category_idx]
            else:
                category = "Other"
        except ValueError:
            category = "Other"

        # Get priority
        print("\nPriority options:")
        print("0. No Priority")
        print("1. Low")
        print("2. Medium")
        print("3. High")
        priority_choice = input("Choose priority (0-3): ")

        priority_map = {"0": "None", "1": "Low", "2": "Medium", "3": "High"}
        priority = priority_map.get(priority_choice, "Medium")

        # Get due date with helper
        print("\nDue date options:")
        print("1. Today")
        print("2. Tomorrow")
        print("3. Next week")
        print("4. Next month")
        print("5. Custom date")

        option = input("Choose option (1-5): ")

        if option == '1':
            due_date = self.datetime_mgr.format_date()
        elif option == '2':
            due_date = self.datetime_mgr.format_relative_date(1)
        elif option == '3':
            due_date = self.datetime_mgr.format_relative_date(7)
        elif option == '4':
            due_date = self.datetime_mgr.format_relative_date(30)
        elif option == '5':
            due_date = input("Enter due date (mm-dd-yyyy or Month day, year): ")
        else:
            due_date = "Not set"

        if not due_date:
            due_date = "Not set"

        # Load tasks
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            tasks = {}
        except Exception as e:
            return False, f"Error reading task data: {e}"

        # Make sure user has a task list
        if self.current_user not in tasks:
            tasks[self.current_user] = []

        # Create task ID (simple sequential numbering)
        task_id = len(tasks[self.current_user]) + 1

        # Get current timestamp
        timestamp = self.datetime_mgr.format_datetime()

        # Create a new task - with all our fields
        new_task = {
            'id': task_id,
            'description': description,
            'category': category,
            'status': 'Pending',
            'priority': priority,
            'due_date': due_date,
            'created_at': timestamp,
            'created_by': self.current_user,
            'completed_at': None,
            'completed_by': None,
            'notes': [],
            'comments': []
        }

        # Add task to user's list
        tasks[self.current_user].append(new_task)

        # Save updated tasks
        try:
            with open(self.tasks_file, 'w') as f:
                json.dump(tasks, f)
        except Exception as e:
            return False, f"Error saving task data: {e}"

        # Log to CSV
        try:
            with open(self.csv_log, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([
                    task_id,
                    self.current_user,
                    description,
                    category,
                    priority,
                    due_date,
                    timestamp,
                    '',  # completed_at (empty)
                    '',  # completed_by (empty)
                    'Pending'
                ])
        except Exception as e:
            # Non-fatal error, just log it
            print(f"Warning: Could not update CSV log: {e}")

        return True, f"Task #{task_id} added & ready to tackle!"

    def edit_task(self, task_id):
        """Edit an existing task.

        Args:
            task_id: ID of task to edit

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to edit task."

        if not self.is_logged_in:
            return False, "Login first before editing tasks."

        # Validate ID format
        try:
            task_id = int(task_id)
        except ValueError:
            return False, "Task ID needs to be a number."

        # Load tasks
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            return False, "No tasks found in the system."
        except Exception as e:
            return False, f"Error reading task data: {e}"

        # Find the task
        task_to_edit = None
        for task in tasks.get(self.current_user, []):
            if task.get('id') == task_id:
                task_to_edit = task
                break

        if not task_to_edit:
            return False, f"Couldn't find Task #{task_id}... check your task list."

        # Show current values and get updates
        print(f"\nEditing Task #{task_id}")
        print(f"Current description: {task_to_edit.get('description')}")
        new_description = input("New description (press Enter to keep current): ")

        print(f"\nCurrent category: {task_to_edit.get('category', 'Other')}")
        print("Category options:")
        for i, category in enumerate(self.categories):
            print(f"{i+1}. {category}")

        category_choice = input(f"Choose new category (1-{len(self.categories)}, press Enter to keep current): ")

        print(f"\nCurrent priority: {task_to_edit.get('priority', 'Medium')}")
        print("Priority options:")
        print("0. No Priority")
        print("1. Low")
        print("2. Medium")
        print("3. High")
        priority_choice = input("Choose new priority (0-3, press Enter to keep current): ")

        print(f"\nCurrent due date: {task_to_edit.get('due_date', 'Not set')}")
        print("Due date options:")
        print("1. Today")
        print("2. Tomorrow")
        print("3. Next week")
        print("4. Next month")
        print("5. Custom date")
        print("Enter: Keep current")

        option = input("Choose option: ")

        if option == '1':
            new_due_date = self.datetime_mgr.format_date()
        elif option == '2':
            new_due_date = self.datetime_mgr.format_relative_date(1)
        elif option == '3':
            new_due_date = self.datetime_mgr.format_relative_date(7)
        elif option == '4':
            new_due_date = self.datetime_mgr.format_relative_date(30)
        elif option == '5':
            new_due_date = input("Enter due date (mm-dd-yyyy or Month day, year): ")
        else:
            new_due_date = ""

        # Update task with new values if provided
        if new_description:
            task_to_edit['description'] = new_description

        if category_choice:
            try:
                category_idx = int(category_choice) - 1
                if 0 <= category_idx < len(self.categories):
                    task_to_edit['category'] = self.categories[category_idx]
            except ValueError:
                pass  # Keep current if invalid

        if priority_choice:
            priority_map = {"0": "None", "1": "Low", "2": "Medium", "3": "High"}
            if priority_choice in priority_map:
                task_to_edit['priority'] = priority_map[priority_choice]

        if new_due_date:
            task_to_edit['due_date'] = new_due_date

        # Add comment about the edit
        timestamp = self.datetime_mgr.format_datetime()
        if 'comments' not in task_to_edit:
            task_to_edit['comments'] = []

        task_to_edit['comments'].append(f"Edited by {self.current_user} on {timestamp}")

        # Save updated tasks
        try:
            with open(self.tasks_file, 'w') as f:
                json.dump(tasks, f)
        except Exception as e:
            return False, f"Error saving task data: {e}"

        return True, f"Task #{task_id} updated successfully!"

    def view_tasks(self):
        """Shows all tasks for current user.

        Returns:
            List of user's tasks
        """
        if not self._check_drive_connected():
            print("Google Drive is not connected. Tasks may not be up to date.")
            return []

        if not self.is_logged_in:
            print("Hold up... you need to login first to see your tasks.")
            return []

        # Load tasks
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            return []
        except Exception as e:
            print(f"Error reading task data: {e}")
            return []

        # Return user's tasks or empty list
        return tasks.get(self.current_user, [])

    def view_filtered_tasks(self, filter_type="all"):
        """View tasks with a specific filter applied.

        Args:
            filter_type: Filter to apply (all, pending, completed, high, overdue)

        Returns:
            Filtered list of tasks
        """
        all_tasks = self.view_tasks()

        if filter_type == "all":
            return all_tasks
        elif filter_type == "pending":
            return [t for t in all_tasks if t.get('status') == 'Pending']
        elif filter_type == "completed":
            return [t for t in all_tasks if t.get('status') == 'Completed']
        elif filter_type == "high":
            return [t for t in all_tasks if t.get('priority') == 'High' and t.get('status') == 'Pending']
        elif filter_type == "overdue":
            # Logic to find overdue tasks based on due_date
            overdue = []
            for task in all_tasks:
                if (task.get('status') == 'Pending' and
                    task.get('due_date') and
                    task.get('due_date') != 'Not set' and
                    self.datetime_mgr.get_date_status(task.get('due_date')) == 'past'):
                    overdue.append(task)
            return overdue
        elif filter_type in self.categories:
            # Filter by category
            return [t for t in all_tasks if t.get('category') == filter_type]
        else:
            return all_tasks

    def search_tasks(self, keyword):
        """Search tasks for a keyword.

        Args:
            keyword: Search term to look for

        Returns:
            List of matching tasks
        """
        all_tasks = self.view_tasks()
        if not keyword:
            return all_tasks

        keyword = keyword.lower()

        return [task for task in all_tasks if
                keyword in task.get('description', '').lower() or
                keyword in task.get('category', '').lower() or
                keyword in str(task.get('id', '')).lower()]

    def mark_completed(self, task_id):
        """Updates task status to Completed.

        Args:
            task_id: ID of task to mark complete

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to update task."

        if not self.is_logged_in:
            return False, "Login first before updating tasks."

        # Make sure task_id is a number
        try:
            task_id = int(task_id)
        except ValueError:
            return False, "Task ID needs to be a number."

        # Load tasks
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            return False, "No tasks found in the system."
        except Exception as e:
            return False, f"Error reading task data: {e}"

        # Check if user has any tasks
        if self.current_user not in tasks:
            return False, "You don't have any tasks yet... add some first!"

        # Find & update the task
        for task in tasks[self.current_user]:
            if task['id'] == task_id:
                if task.get('status') == 'Completed':
                    return False, f"Task #{task_id} is already completed... no need to update."

                # Current timestamp
                timestamp = self.datetime_mgr.format_datetime()

                # Update task
                task['status'] = 'Completed'
                task['completed_at'] = timestamp
                task['completed_by'] = self.current_user

                # Add completion comment
                if 'comments' not in task:
                    task['comments'] = []

                task['comments'].append(f"Completed by {self.current_user} on {timestamp}")

                # Save updated tasks
                try:
                    with open(self.tasks_file, 'w') as f:
                        json.dump(tasks, f)
                except Exception as e:
                    return False, f"Error saving task data: {e}"

                # Update CSV log
                self._update_csv_completion(task_id, timestamp)

                return True, f"Nice work! Task #{task_id} marked as completed."

        return False, f"Couldn't find Task #{task_id}... check your task list."

    def _update_csv_completion(self, task_id, timestamp):
        """Updates the CSV log when a task is completed.

        Args:
            task_id: ID of the completed task
            timestamp: Completion timestamp
        """
        if not os.path.exists(self.csv_log):
            print("CSV log not found. Skipping update.")
            return

        # Read existing CSV
        rows = []
        try:
            with open(self.csv_log, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                header = next(reader)  # Skip header
                rows.append(header)  # Keep the header
                for row in reader:
                    if len(row) >= 9 and int(row[0]) == task_id and row[1] == self.current_user:
                        # Update the completion info
                        row[7] = timestamp  # completed_at
                        row[8] = self.current_user  # completed_by
                        row[9] = 'Completed'  # status
                    rows.append(row)
        except Exception as e:
            print(f"Warning: Could not read CSV log: {e}")
            return

        # Write back updated CSV
        try:
            with open(self.csv_log, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(rows)
        except Exception as e:
            print(f"Warning: Could not write CSV log: {e}")

    def set_priority(self, task_id, priority):
        """Sets priority level for a task.

        Args:
            task_id: ID of task to update
            priority: New priority level (High, Medium, Low)

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to update task."

        if not self.is_logged_in:
            return False, "Login first before updating tasks."

        # Validate inputs
        try:
            task_id = int(task_id)
        except ValueError:
            return False, "Task ID needs to be a number."

        if priority not in ["None", "Low", "Medium", "High"]:
            return False, "Priority must be None, Low, Medium, or High."

        # Load tasks
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            return False, "No tasks found in the system."
        except Exception as e:
            return False, f"Error reading task data: {e}"

        # Find & update task
        for task in tasks.get(self.current_user, []):
            if task['id'] == task_id:
                task['priority'] = priority

                # Add comment
                if 'comments' not in task:
                    task['comments'] = []

                timestamp = self.datetime_mgr.format_datetime()
                task['comments'].append(f"Priority set to {priority} by {self.current_user} on {timestamp}")

                # Save updated tasks
                try:
                    with open(self.tasks_file, 'w') as f:
                        json.dump(tasks, f)
                except Exception as e:
                    return False, f"Error saving task data: {e}"

                return True, f"Task #{task_id} is now {priority} priority."

        return False, f"Couldn't find Task #{task_id}... check your task list."

    def set_due_date(self, task_id, due_date):
        """Sets a due date for a task.

        Args:
            task_id: ID of task to update
            due_date: Date string in any reasonable format

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to update task."

        if not self.is_logged_in:
            return False, "Login first before updating tasks."

        # Validate inputs
        try:
            task_id = int(task_id)
        except ValueError:
            return False, "Task ID needs to be a number."

        # Load tasks
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            return False, "No tasks found in the system."
        except Exception as e:
            return False, f"Error reading task data: {e}"

        # Find & update task
        for task in tasks.get(self.current_user, []):
            if task['id'] == task_id:
                task['due_date'] = due_date

                # Add comment
                if 'comments' not in task:
                    task['comments'] = []

                timestamp = self.datetime_mgr.format_datetime()
                task['comments'].append(f"Due date set to {due_date} by {self.current_user} on {timestamp}")

                # Save updated tasks
                try:
                    with open(self.tasks_file, 'w') as f:
                        json.dump(tasks, f)
                except Exception as e:
                    return False, f"Error saving task data: {e}"

                return True, f"Task #{task_id} is now due on {due_date}."

        return False, f"Couldn't find Task #{task_id}... check your task list."

    def add_note_to_task(self, task_id, note):
        """Add a custom note to a task.

        Args:
            task_id: ID of the task
            note: Note text to add

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to update task."

        if not self.is_logged_in:
            return False, "Login first before updating tasks."

        if not note:
            return False, "Note can't be empty."

        # Validate inputs
        try:
            task_id = int(task_id)
        except ValueError:
            return False, "Task ID needs to be a number."

        # Load tasks
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            return False, "No tasks found in the system."
        except Exception as e:
            return False, f"Error reading task data: {e}"

        # Find task
        for task in tasks.get(self.current_user, []):
            if task['id'] == task_id:
                if 'notes' not in task:
                    task['notes'] = []

                timestamp = self.datetime_mgr.format_datetime()
                task['notes'].append({
                    'text': note,
                    'added_by': self.current_user,
                    'added_at': timestamp
                })

                # Add to comments too
                if 'comments' not in task:
                    task['comments'] = []

                task['comments'].append(f"Note added by {self.current_user} on {timestamp}")

                # Save updated tasks
                try:
                    with open(self.tasks_file, 'w') as f:
                        json.dump(tasks, f)
                except Exception as e:
                    return False, f"Error saving task data: {e}"

                return True, f"Note added to Task #{task_id}!"

        return False, f"Couldn't find Task #{task_id}... check your task list."

    def delete_task(self, task_id):
        """Removes a task completely.

        Args:
            task_id: ID of task to delete

        Returns:
            Success flag & message
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to delete task."

        if not self.is_logged_in:
            return False, "Login first before managing tasks."

        # Make sure task_id is a number
        try:
            task_id = int(task_id)
        except ValueError:
            return False, "Task ID needs to be a number."

        # Load tasks
        try:
            with open(self.tasks_file, 'r') as f:
                tasks = json.load(f)
        except json.JSONDecodeError:
            return False, "No tasks found in the system."
        except Exception as e:
            return False, f"Error reading task data: {e}"

        # Check if user has any tasks
        if self.current_user not in tasks:
            return False, "You don't have any tasks yet!"

        # Find & delete the task
        user_tasks = tasks[self.current_user]
        for i, task in enumerate(user_tasks):
            if task.get('id') == task_id:
                del user_tasks[i]

                # Save updated tasks
                try:
                    with open(self.tasks_file, 'w') as f:
                        json.dump(tasks, f)
                except Exception as e:
                    return False, f"Error saving task data: {e}"

                return True, f"Task #{task_id} is gone for good!"

        return False, f"Couldn't find Task #{task_id}... check your task list."

    def get_task_stats(self):
        """Get stats about user's tasks.

        Returns:
            Dictionary with task statistics
        """
        if not self._check_drive_connected():
            print("Google Drive is not connected. Stats may not be up to date.")
            return {}

        if not self.is_logged_in:
            return {}

        tasks = self.view_tasks()

        # Count tasks by status
        pending_count = sum(1 for task in tasks if task.get('status') == 'Pending')
        completed_count = sum(1 for task in tasks if task.get('status') == 'Completed')

        # Count by priority
        high_priority = sum(1 for task in tasks if task.get('priority') == 'High' and task.get('status') == 'Pending')

        # Count by category
        categories = {}
        for task in tasks:
            cat = task.get('category', 'Other')
            if cat not in categories:
                categories[cat] = 0
            categories[cat] += 1

        # Count overdue tasks
        overdue = 0
        for task in tasks:
            if (task.get('status') == 'Pending' and
                task.get('due_date') and
                task.get('due_date') != 'Not set' and
                self.datetime_mgr.get_date_status(task.get('due_date')) == 'past'):
                overdue += 1

        # Tasks completed recently
        completed_week = 0
        week_ago_str = self.datetime_mgr.format_datetime(
            self.datetime_mgr.now() - timedelta(days=7)
        )

        for task in tasks:
            if task.get('status') == 'Completed' and task.get('completed_at'):
                try:
                    if task.get('completed_at') > week_ago_str:
                        completed_week += 1
                except:
                    pass

        # Return stats dictionary
        return {
            'total': len(tasks),
            'pending': pending_count,
            'completed': completed_count,
            'high_priority': high_priority,
            'overdue': overdue,
            'completed_week': completed_week,
            'categories': categories
        }

    def export_tasks_to_csv(self, filename=None):
        """Export user's tasks to a CSV file.

        Args:
            filename: Optional custom filename

        Returns:
            Success flag & message, and path to CSV file
        """
        # Check if drive is connected
        if not self._check_drive_connected():
            return False, "Google Drive is not connected. Unable to export tasks.", None

        if not self.is_logged_in:
            return False, "You need to login first to export tasks.", None

        tasks = self.view_tasks()
        if not tasks:
            return False, "No tasks to export!", None

        if not filename:
            timestamp = self.datetime_mgr.now().strftime('%Y%m%d_%H%M%S')
            filename = f"{self.current_user}_tasks_{timestamp}.csv"

        filepath = os.path.join(self.base_dir, filename)

        try:
            with open(filepath, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                # Write header
                writer.writerow(['ID', 'Description', 'Category', 'Status', 'Priority',
                                'Due Date', 'Created At', 'Completed At'])

                # Write task data
                for task in tasks:
                    writer.writerow([
                        task.get('id', ''),
                        task.get('description', ''),
                        task.get('category', 'Other'),
                        task.get('status', ''),
                        task.get('priority', ''),
                        task.get('due_date', ''),
                        task.get('created_at', ''),
                        task.get('completed_at', '')
                    ])

            return True, f"Tasks exported to {filename}!", filepath
        except Exception as e:
            return False, f"Export failed: {e}", None

def show_python_logo():
    """Display a small Python logo at the top of the page."""
    logo_html = """
    <div style="text-align: center;">
        <img src="https://www.python.org/static/community_logos/python-logo-generic.svg"
             width="100" alt="Python Logo">
    </div>
    """
    try:
        display.display(display.HTML(logo_html))
    except:
        # If displaying HTML fails, use ASCII art fallback with Python colors
        python_blue = "\033[34m"    # Blue color
        python_yellow = "\033[33m"  # Yellow color
        reset_color = "\033[0m"     # Reset to default color

        print(f"""
        {python_blue}          _____
                |  __ \\
                | |__) |{python_yellow}   _ | |_| |__   ___  _ __
                |  ___/ | | || __| '_ \\ / _ \\| '_ \\
                | |   | |_| || |_| | | | (_) | | | |
                |_|    \\__, | \\__|_| |_|\\___/|_| |_|
                        __/ |
                       |___/{reset_color}
        """)

def print_header(title):
    """Shows a nice formatted header.

    Args:
        title: Text to display in the header
    """
    # Show Python logo at the top
    show_python_logo()

    width = 60  # Increased width for better display
    print("\n" + "=" * width)
    print(f"{title:^{width}}")
    print("=" * width + "\n")

def colorize(text, color_code):
    """Add ANSI color to text for Colab display.

    Args:
        text: Text to colorize
        color_code: ANSI color code

    Returns:
        Colorized text
    """
    return f"\033[{color_code}m{text}\033[0m"

def print_message(message, success=True):
    """Displays feedback messages to the user.

    Args:
        message: Message to show
        success: Whether it's good news or bad news
    """
    if success:
        status = colorize("SUCCESS", "32")  # Green
    else:
        status = colorize("PROBLEM", "31")  # Red

    print(f"\n[{status}] {message}\n")

def get_status_icon(status, priority="Medium"):
    """Get an icon representing task status and priority.

    Args:
        status: Task status
        priority: Task priority

    Returns:
        Emoji icon for status
    """
    if status == "Completed":
        return "✅"
    elif priority == "High":
        return "🔴"
    elif priority == "Medium":
        return "🟠"
    elif priority == "Low":
        return "🟡"
    else:
        return "⏳"

def display_tasks(tasks, filter_type="all"):
    """Shows tasks in a nice readable format.

    Args:
        tasks: List of task dictionaries
        filter_type: Optional filter type being used
    """
    if not tasks:
        if filter_type == "all":
            print_message("No tasks found yet... time to add some!", False)
        else:
            print_message(f"No tasks found matching the '{filter_type}' filter.", False)
        return

    filter_label = ""
    if filter_type != "all":
        filter_label = f" ({filter_type.capitalize()})"

    print_header(f"Your Task List{filter_label}")

    # Optional: Hide completed tasks for default view
    # if filter_type == "all":
    #     tasks = [t for t in tasks if t.get('status') != 'Completed']

    # Sort tasks: pending high priority first, then others, then completed
    sorted_tasks = sorted(tasks, key=lambda t: (
        0 if t.get('status') == 'Pending' and t.get('priority') == 'High' else
        1 if t.get('status') == 'Pending' else
        2
    ))

    print(f"{'ID':^5} | {'Status':^4} | {'Category':^10} | {'Priority':^8} | {'Due Date':^15} | {'Description':^30}")
    print("-" * 85)

    # Get datetime manager from the first task manager instance
    # (This is a bit of a hack, but it works for this demo)
    dt_mgr = DateTimeManager()

    for task in sorted_tasks:
        # Safely get values with defaults
        task_id = task.get('id', 'N/A')
        status = task.get('status', 'Unknown')
        description = task.get('description', 'No description')
        priority = task.get('priority', 'Medium')
        due_date = task.get('due_date', 'Not set')
        category = task.get('category', 'Other')

        # Get the date status
        date_status = dt_mgr.get_date_status(due_date)

        # Get status icon
        icon = get_status_icon(status, priority)

        # Trim long descriptions to fit nicely
        if len(description) > 27:
            description = description[:27] + "..."

        # Format the row
        row = f"{task_id:^5} | {icon:^4} | {category:^10} | {priority:^8} | {due_date:^15} | {description:<30}"

        # Apply coloring based on status and due date
        if status == 'Completed':
            # Green for completed tasks
            print(colorize(row, "32"))
        elif date_status == 'past':
            # Red for overdue tasks
            print(colorize(row, "31"))
        elif date_status == 'today':
            # Yellow for tasks due today
            print(colorize(row, "33"))
        elif date_status == 'future':
            # Green for future tasks
            print(colorize(row, "32"))
        elif priority == 'High':
            # Bold red for high priority with no due date
            print(colorize(row, "1;31"))
        else:
            # Normal text for other tasks
            print(row)

    print("-" * 85)
    print(f"Showing {len(sorted_tasks)} task(s)")

def display_task_details(task):
    """Shows all details for a single task.

    Args:
        task: Task dictionary
    """
    if not task:
        print_message("Task not found!", False)
        return

    # Get status icon
    icon = get_status_icon(task.get('status'), task.get('priority'))

    print_header(f"Task #{task.get('id')} Details")
    print(f"{icon} Description: {task.get('description')}")
    print(f"Category: {task.get('category', 'Other')}")
    print(f"Status: {colorize(task.get('status'), '32' if task.get('status') == 'Completed' else '37')}")
    print(f"Priority: {colorize(task.get('priority'), '31' if task.get('priority') == 'High' else '33')}")
    print(f"Due Date: {task.get('due_date')}")
    print(f"Created At: {task.get('created_at')}")
    print(f"Created By: {task.get('created_by', 'Unknown')}")

    if task.get('status') == 'Completed':
        print(f"Completed At: {task.get('completed_at')}")
        print(f"Completed By: {task.get('completed_by', 'Unknown')}")

    # Display notes if any
    if task.get('notes'):
        print("\nNotes:")
        for note in task.get('notes', []):
            note_time = note.get('added_at', '')
            note_user = note.get('added_by', 'Unknown')
            note_text = note.get('text', '')
            print(f"- [{note_time}] {note_user}: {note_text}")

    # Display comments if any
    if task.get('comments'):
        print("\nActivity History:")
        for comment in task.get('comments', []):
            print(f"- {comment}")

    print("-" * 60)

def display_stats(stats):
    """Display task statistics in a nice format.

    Args:
        stats: Dictionary with task statistics
    """
    if not stats:
        return

    print_header("Your Task Statistics")

    print(f"Total Tasks: {stats['total']}")
    print(f"Pending: {colorize(str(stats['pending']), '33')} | Completed: {colorize(str(stats['completed']), '32')}")

    # Show completion rate as percentage if there are tasks
    if stats['total'] > 0:
        completion_rate = (stats['completed'] / stats['total']) * 100
        print(f"Completion Rate: {completion_rate:.1f}%")

    print(f"High Priority Tasks: {colorize(str(stats['high_priority']), '31')}")
    print(f"Overdue Tasks: {colorize(str(stats['overdue']), '31')}")
    print(f"Completed in Last 7 Days: {colorize(str(stats['completed_week']), '32')}")

    # Show task counts by category
    if stats['categories']:
        print("\nTasks by Category:")
        for category, count in stats['categories'].items():
            print(f"- {category}: {count}")

    print("-" * 60)

def run_app():
    """Fires up the Task Manager & keeps it running."""
    # Create TaskManager instance (using Denver timezone by default)
    tm = TaskManager(timezone='America/Denver')

    while True:
        # Auth menu if not logged in
        if not tm.is_logged_in:
            print_header("Task Manager w/Authentication")

            # Check Google Drive connection and show status
            drive_connected = tm._check_drive_connected()
            drive_status = "Connected to Google Drive ✓" if drive_connected else "Google Drive not connected ✗"
            print(f"Google Drive Status: {colorize(drive_status, '32' if drive_connected else '31')}")

            print("\n1. Login")
            print("2. Register")
            print("3. Connect to Google Drive") # New option
            print("4. Exit")
            print("\nKeyboard shortcuts: [l]ogin, [r]egister, [d]rive, [x]exit")

            choice = input("\nWhat'll it be? ").lower()

            if choice in ['1', 'l', 'login']:
                if not drive_connected:
                    print_message("You need to connect to Google Drive before logging in.", False)
                    print("Please select option 3 to connect to Google Drive first.")
                    continue

                username = input("Username: ")
                password = getpass.getpass("Password: ")

                success, message = tm.login(username, password)
                print_message(message, success)

            elif choice in ['2', 'r', 'register']:
                if not drive_connected:
                    print_message("You need to connect to Google Drive before registering.", False)
                    print("Please select option 3 to connect to Google Drive first.")
                    continue

                username = input("Choose a username: ")
                password = getpass.getpass("Create a password: ")
                confirm = getpass.getpass("Confirm that password: ")

                if password != confirm:
                    print_message("Passwords don't match! Let's try that again...", False)
                    continue

                success, message = tm.register(username, password)
                print_message(message, success)

            elif choice in ['3', 'd', 'drive']:
                # Force a new connection attempt
                tm._connect_to_google_drive(force_auth=True)

            elif choice in ['4', 'x', 'exit']:
                print_message("Thanks for using the Task Manager. Have a great day!")
                break

            else:
                print_message("That's not a valid option... try 1, 2, 3, or 4.", False)

        # Task menu for logged in users
        else:
            # Get task stats
            stats = tm.get_task_stats()

            # Show welcome with stats summary
            print_header(f"Task Manager - Welcome, {tm.current_user}!")

            if stats:
                print(f"You have {colorize(str(stats['pending']), '33')} pending tasks " +
                      f"({colorize(str(stats['high_priority']), '31')} high priority, " +
                      f"{colorize(str(stats['overdue']), '31')} overdue)")

            print("\nWhat would you like to do?")
            print("1. Add a Task")
            print("2. View Tasks")
            print("3. Search Tasks")
            print("4. View Filtered Tasks")
            print("5. Mark Task Complete")
            print("6. Edit Task")
            print("7. Add Note to Task")
            print("8. View Task Details")
            print("9. Task Statistics")
            print("10. Export Tasks to CSV")
            print("0. Logout")

            print("\nKeyboard shortcuts: [a]dd, [v]iew, [s]earch, [f]ilter, [c]omplete, [e]dit, [q]uit")

            choice = input("\nEnter your choice: ").lower()

            if choice in ['1', 'a', 'add']:
                success, message = tm.add_task()
                print_message(message, success)

            elif choice in ['2', 'v', 'view']:
                tasks = tm.view_tasks()
                display_tasks(tasks)
                input("\nPress Enter when you're done looking...")

            elif choice in ['3', 's', 'search']:
                keyword = input("Enter search term: ")
                matching_tasks = tm.search_tasks(keyword)
                display_tasks(matching_tasks, f"search: {keyword}")
                input("\nPress Enter when you're done looking...")

            elif choice in ['4', 'f', 'filter']:
                print("\nFilter options:")
                print("1. All tasks")
                print("2. Pending tasks")
                print("3. Completed tasks")
                print("4. High priority tasks")
                print("5. Overdue tasks")
                print("6. Filter by category")

                filter_choice = input("Choose filter (1-6): ")

                filter_map = {
                    "1": "all",
                    "2": "pending",
                    "3": "completed",
                    "4": "high",
                    "5": "overdue"
                }

                if filter_choice == "6":
                    print("\nCategories:")
                    for i, category in enumerate(tm.categories):
                        print(f"{i+1}. {category}")

                    cat_choice = input(f"Choose category (1-{len(tm.categories)}): ")
                    try:
                        cat_idx = int(cat_choice) - 1
                        if 0 <= cat_idx < len(tm.categories):
                            filter_type = tm.categories[cat_idx]
                        else:
                            filter_type = "all"
                    except ValueError:
                        filter_type = "all"
                else:
                    filter_type = filter_map.get(filter_choice, "all")

                filtered_tasks = tm.view_filtered_tasks(filter_type)
                display_tasks(filtered_tasks, filter_type)
                input("\nPress Enter when you're done looking...")

            elif choice in ['5', 'c', 'complete']:
                tasks = tm.view_tasks()
                display_tasks(tasks)

                if tasks:
                    task_id = input("Which task did you complete? (Enter ID #): ")
                    success, message = tm.mark_completed(task_id)
                    print_message(message, success)

                    if success:
                        print("\n🎉 Congratulations on completing your task! 🎉")

            elif choice in ['6', 'e', 'edit']:
                tasks = tm.view_tasks()
                display_tasks(tasks)

                if tasks:
                    task_id = input("Which task do you want to edit? (Enter ID #): ")
                    success, message = tm.edit_task(task_id)
                    print_message(message, success)

            elif choice == '7':
                tasks = tm.view_tasks()
                display_tasks(tasks)

                if tasks:
                    task_id = input("Which task do you want to add a note to? (Enter ID #): ")
                    note = input("Enter your note: ")
                    success, message = tm.add_note_to_task(task_id, note)
                    print_message(message, success)

            elif choice == '8':
                tasks = tm.view_tasks()
                display_tasks(tasks)

                if tasks:
                    task_id = input("Which task do you want to view in detail? (Enter ID #): ")
                    try:
                        task_id = int(task_id)
                        task = next((t for t in tasks if t.get('id') == task_id), None)
                        if task:
                            display_task_details(task)
                            input("\nPress Enter to continue...")
                        else:
                            print_message(f"Task #{task_id} not found!", False)
                    except ValueError:
                        print_message("Task ID must be a number!", False)

            elif choice == '9':
                # Display detailed stats
                display_stats(stats)
                input("\nPress Enter to continue...")

            elif choice == '10':
                success, message, filepath = tm.export_tasks_to_csv()
                print_message(message, success)
                if success:
                    print(f"File saved to: {filepath}")
                input("\nPress Enter to continue...")

            elif choice in ['0', 'q', 'quit', 'logout']:
                message = tm.logout()
                print_message(message)

            else:
                print_message("That's not a valid option... please try again.", False)

# Let's get this show on the road
if __name__ == "__main__":
    try:
        print_header("Task Manager w/Authentication")
        print("Your tasks, your way")
        print("---------------------------------------------------")
        print("This app will store your tasks securely in Google Drive.")
        print("No question, you'll be able to access them from anywhere!")

        run_app()
    except KeyboardInterrupt:
        print("\nLooks like you're heading out. Have a good one!")
    except Exception as e:
        print(f"\nHmm, something went wrong: {e}")
        print("The app needs to close... try again when you can.")


               Task Manager w/Authentication                

Your tasks, your way
---------------------------------------------------
This app will store your tasks securely in Google Drive.
No question, you'll be able to access them from anywhere!



               Task Manager w/Authentication                

Google Drive Status: Google Drive not connected ✗

1. Login
2. Register
3. Connect to Google Drive
4. Exit

Keyboard shortcuts: [l]ogin, [r]egister, [d]rive, [x]exit

What'll it be? 3
Connecting to Google Drive...
Please follow the authentication steps in the popup window if prompted.
Mounted at /content/drive
Successfully connected to Google Drive!



               Task Manager w/Authentication                

Google Drive Status: Connected to Google Drive ✓

1. Login
2. Register
3. Connect to Google Drive
4. Exit

Keyboard shortcuts: [l]ogin, [r]egister, [d]rive, [x]exit

What'll it be? 1
Username: DLantz
Password: ··········

[SUCCESS] Welcome back, DLantz! Your tasks are ready.




              Task Manager - Welcome, DLantz!               

You have 3 pending tasks (3 high priority, 0 overdue)

What would you like to do?
1. Add a Task
2. View Tasks
3. Search Tasks
4. View Filtered Tasks
5. Mark Task Complete
6. Edit Task
7. Add Note to Task
8. View Task Details
9. Task Statistics
10. Export Tasks to CSV
0. Logout

Keyboard shortcuts: [a]dd, [v]iew, [s]earch, [f]ilter, [c]omplete, [e]dit, [q]uit

Enter your choice: 2



                       Your Task List                       

 ID   | Status |  Category  | Priority |    Due Date     |          Description          
-------------------------------------------------------------------------------------
  4   |  🔴   |   Other    |   High   |  May 13, 2025   | Python Basics Class           
  7   |  🔴   |   Other    |   High   |  May 16, 2025   | Python - Course_End_Project   
  5   |  🔴   |   Study    |   High   |  May 16, 2025   | Python Quiz                   
  3   |  ✅   |   Study    |   High   |  May 12, 2025   | Python Basics Class           
  6   |  ✅   |   Other    |   High   |  May 11, 2025   | Python Basics Class           
-------------------------------------------------------------------------------------
Showing 5 task(s)

Looks like you're heading out. Have a good one!
